<a href="https://colab.research.google.com/github/harsha-bsm/Topic-Modelling-using-LDA-and-NMF-on-SamsungM30s-reviewsflipkart/blob/master/Preprocessing_and_Modelling_LDA_SAMSUNGM30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import numpy as np
import pandas as pd

In [ ]:
data=pd.read_csv("samsungdata.csv")
dataoriginal=pd.read_csv("samsungdata.csv")

In [ ]:
data.head()

,Unnamed: 0,Model,rating,user,time_of_review,review,title,user_address,user_type
0,0,Samsung Galaxy M30s Blue 128 Gb Reviews: Lates...,5,Aryan Patil,2months ago,Bokeh mode is very excellent than any other ha...,Wonderful,Latur District,Certified Buyer
1,1,Samsung Galaxy M30s Blue 128 Gb Reviews: Lates...,5,Flipkart Customer,5months ago,Just awsome product the picture quality is goo...,Great product,North Twenty Four Parganas District,Certified Buyer
2,2,Samsung Galaxy M30s Blue 128 Gb Reviews: Lates...,4,Suresh Kumar,5months ago,Samsung galaxy m30s is most useful.But Samsung...,Nice product,New Delhi,Certified Buyer
3,3,Samsung Galaxy M30s Blue 128 Gb Reviews: Lates...,1,Rohit Purba,6months ago,Not worth for me. i buy for my samsung tv this...,Utterly Disappointed,Rajpura,Certified Buyer
4,4,Samsung Galaxy M30s Blue 128 Gb Reviews: Lates...,5,Flipkart Customer,5months ago,"BSOLUTELY BEST PHONE, GO FOR THIS PHONELET ME ...",Highly recommended,Muzaffarpur,Certified Buyer


In [ ]:
pip install emot 

In [ ]:
####### Apply emoticons and emojis

from emot.emo_unicode import UNICODE_EMO, EMOTICONS # contains english text of emojis and emoticons in dict format 
def convert_emojis(review):
  for x in review:
    if x in UNICODE_EMO.keys():
      review=review.replace(x,"")
  return review

In [ ]:
#converting all basic contracted foms  
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [ ]:
#removing special characters and tabs ,spaces,new lines etc..
def removecharacters(review):
  review= re.sub('[^A-Za-z0-9]+',' ',review)  #anything exept numbers and alphabets, replace them with space
  review=re.sub(r"\n"," ",review)  #new lines into space
  review=re.sub(r"\t"," ",review)  #tabs into space
  review=re.sub(r"\v"," ",review)  #vertical tab into space
  review=re.sub(r"\s"," ",review)   #all extra spaces into single space
  return review  


In [ ]:
# https://gist.github.com/sebleier/554280
# we are removing the words from the stop words list: 'no', 'nor', 'not'
# <br /><br /> ==> after the above steps, we are getting "br br"
# we are including them into stop words list
# instead of <br /> if we have <br/> these tags would have revmoved in the 1st step
# ignore words like won't, haven't etc.. We have already taken care of them

stopword= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't","not","phone","product","mobile","np"])

In [ ]:
from nltk.corpus import stopwords
import nltk

In [ ]:
def removestopwords(review):
  return " ".join([word for word in review.split(" ") if not word in stopword])


In [ ]:
data["review"]=data["review"].apply(lambda x:convert_emojis(x))
data["review"]=data["review"].apply(lambda x:x.lower()) 
data["review"]=data["review"].apply(lambda x:decontracted(x))
data["review"]=data["review"].apply(lambda x:removecharacters(x))
data["review"]=pd.DataFrame(data["review"].apply(lambda x:removestopwords(x)))

In [ ]:
data["review"]

0      bokeh mode excellent handset budget battery ca...
1      awsome picture quality good battery backup aws...
2      samsung galaxy m30s useful samsung improve sel...
3      worth buy samsung tv live cast smartthins app ...
4      bsolutely best go phonelet clear features1 cam...
                             ...                        
293                                                 okay
294                           super style good look nice
295                                                 good
296                                                 good
297                                                 good
Name: review, Length: 298, dtype: object

In [ ]:
data["review"].to_csv("samsung_preprocessed.csv")

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import ldamodel
import gensim.corpora
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
#https://scikit-learn.org/stable/auto_examples/applications/plot_topics_extraction_with_nmf_lda.html#sphx-glr-auto-examples-applications-plot-topics-extraction-with-nmf-lda-py
def print_top_words(model, feature_names, n_top_words):
  topics_cluster=[]
  for topic_idx, topic in enumerate(model.components_):
      message = "Topic #%d: " % topic_idx
      message += ",".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
      topics_cluster.append([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
  return topics_cluster
  

### BOW and LDA 

In [ ]:
bow=CountVectorizer(ngram_range=(1,2),min_df=4,max_df=0.95)
bow_review=bow.fit_transform(data["review"])
df_bow=pd.DataFrame(bow_review.toarray(),columns=bow.get_feature_names())
lda = LatentDirichletAllocation(n_components=4,random_state=0)
lda.fit_transform(df_bow)
topics=print_top_words(lda,bow.get_feature_names(),5)

In [ ]:
lda.fit_transform(df_bow)

array([[0.18804108, 0.61229913, 0.02127953, 0.17838026],
       [0.02337478, 0.92847277, 0.02339064, 0.02476181],
       [0.1755224 , 0.56670166, 0.23981901, 0.01795692],
       ...,
       [0.12500307, 0.62499174, 0.12500256, 0.12500263],
       [0.12500307, 0.62499174, 0.12500256, 0.12500263],
       [0.12500307, 0.62499174, 0.12500256, 0.12500263]])

In [ ]:
topic=["topic_"+str(i) for i  in range(0,lda.n_components)]
df_doc_topic=pd.DataFrame(lda.fit_transform(df_bow),columns=topic)
dominant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic["dominant_topic"] = dominant_topic
df_doc_topic["review"]=dataoriginal["review"]
df_doc_topic["dominant_topic_keywords"]=[topics[i]  for i in dominant_topic]
df_doc_topic

,topic_0,topic_1,topic_2,topic_3,dominant_topic,review,dominant_topic_keywords
0,0.188041,0.612299,0.021280,0.178380,1,Bokeh mode is very excellent than any other ha...,"[good, camera, battery, performance, quality]"
1,0.023375,0.928473,0.023391,0.024762,1,Just awsome product the picture quality is goo...,"[good, camera, battery, performance, quality]"
2,0.175522,0.566702,0.239819,0.017957,1,Samsung galaxy m30s is most useful.But Samsung...,"[good, camera, battery, performance, quality]"
3,0.025216,0.025357,0.923147,0.026281,2,Not worth for me. i buy for my samsung tv this...,"[nice, super, samsung, buy, delivery]"
4,0.209835,0.539017,0.207466,0.043682,1,"BSOLUTELY BEST PHONE, GO FOR THIS PHONELET ME ...","[good, camera, battery, performance, quality]"
...,...,...,...,...,...,...,...
293,0.250000,0.250000,0.250000,0.250000,0,okay,"[best, flipkart, superb, xcellent, excellent]"
294,0.062505,0.312429,0.562291,0.062775,2,Super style and good look nice product,"[nice, super, samsung, buy, delivery]"
295,0.125003,0.624992,0.125003,0.125003,1,good,"[good, camera, battery, performance, quality]"
296,0.125003,0.624992,0.125003,0.125003,1,good,"[good, camera, battery, performance, quality]"


#### LDA on topics obtained

In [ ]:
arr=np.array(print_top_words(lda,bow.get_feature_names(),5)).reshape(1,-1)[0]
bow=CountVectorizer(ngram_range=(1,2))
bow_review=bow.fit_transform(arr)
df_bow=pd.DataFrame(bow_review.toarray(),columns=bow.get_feature_names())
lda = LatentDirichletAllocation(n_components=4,random_state=0)
lda.fit_transform(df_bow)
print_top_words(lda,bow.get_feature_names(),4)


[['buy', 'online', 'performance', 'buy online'],
 ['good', 'best', 'nice', 'xcellent'],
 ['battery', 'camera', 'samsung', 'wesome'],
 ['thanks', 'price', 'hangs', 'awsome']]

### TFIDF AND LDA

In [ ]:
tfidf=TfidfVectorizer(ngram_range=(1,2))
tfidf_words=tfidf.fit_transform(data["review"])
words=tfidf.get_feature_names()
df_bow=pd.DataFrame(tfidf_words.toarray(),columns=words)
lda = LatentDirichletAllocation(n_components=10,random_state=0)
lda.fit_transform(tfidf_words)
print_top_words(lda,words,8)

[['money',
  'value money',
  'value',
  'okay',
  'super nice',
  'flipkart',
  'wesome value',
  'things good'],
 ['good',
  'nice',
  'superb',
  'battery',
  'battery backup',
  'camera',
  'backup',
  'backup good'],
 ['xcellent',
  'awsm',
  'waysome',
  'utstanding',
  'nice awsm',
  'easy',
  'smooth easy',
  'battery'],
 ['loved',
  'rattan',
  'worth money',
  'worth',
  'money',
  'camera battery',
  'good one',
  'nice cell'],
 ['love',
  'thanks',
  'nice',
  'display',
  'excellent',
  'super',
  'xcellent',
  'flipkart'],
 ['wesome', 'bad', 'comments', 'camara', 'poor', 'use', 'expected', 'thank'],
 ['super',
  'smartphone',
  'superrrrbbbbb',
  'sm',
  'buy',
  'flipkart',
  'nice products',
  'xcellent smartphone'],
 ['ok', 'price', 'suprrrrrrrrr', 'wsm', 'best', 'set', 'like', 'good price'],
 ['camera',
  'fantastic',
  'goods',
  'fresh',
  'ear',
  'good',
  'wesome',
  'super camera'],
 ['best',
  'awesome',
  'good',
  'samsung',
  'good samsung',
  'great',
  'wo

##### LDA on topics obtained 

In [ ]:
arr=np.array(print_top_words(lda,words,5)).reshape(1,-1)[0]
tfidf=TfidfVectorizer(ngram_range=(1,1))
tfidf_words=tfidf.fit_transform(arr)
words=tfidf.get_feature_names()
df_tfidf=pd.DataFrame(tfidf_words.toarray(),columns=words)
lda = LatentDirichletAllocation(n_components=4,random_state=0)
lda.fit_transform(df_tfidf)
print_top_words(lda,words,4)


[['nice', 'love', 'buy', 'camera'],
 ['money', 'best', 'value', 'rattan'],
 ['good', 'super', 'samsung', 'nice'],
 ['worth', 'awsm', 'excellent', 'superrrrbbbbb']]

### NMF - TFIDF

In [ ]:
from sklearn.decomposition import NMF, LatentDirichletAllocation

In [ ]:
tfidf=TfidfVectorizer(ngram_range=(1,1))
tfidf_words=tfidf.fit_transform(data["review"])
words=tfidf.get_feature_names()
df_lda=pd.DataFrame(tfidf_words.toarray(),columns=words)


In [ ]:
nmf = NMF(n_components=10, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf_words)

print_top_words(nmf,words,6)

[['good', 'one', 'normal', 'choic', 'isplay', 'choice'],
 ['nice', 'awsm', 'phones', 'samsang', 'cell', 'osm'],
 ['super', 'quality', 'camera', 'also', 'performance', 'love'],
 ['wesome', 'outstanding', 'perfomance', 'loving', 'slow', 'gaming'],
 ['best', 'samsung', 'price', 'loved', 'range', 'buy'],
 ['battery', 'camera', 'display', 'backup', 'poor', 'back'],
 ['superb', 'expected', 'great', 'got', 'galaxy', 'tq'],
 ['xcellent', 'flipkart', 'smartphone', 'love', 'excellent', 'thanks'],
 ['money', 'awesome', 'value', 'much', 'worth', 'delivery'],
 ['bad', 'ok', 'thank', 'worst', 'working', 'hangs']]

#### NMF on topics obtained

In [ ]:
arr=np.array(print_top_words(nmf,words,5)).reshape(1,-1)[0]

In [ ]:
tfidf=TfidfVectorizer(ngram_range=(1,2))
tfidf_words=tfidf.fit_transform(arr)
words=tfidf.get_feature_names()
df_lda=pd.DataFrame(tfidf_words.toarray(),columns=words)
nmf = NMF(n_components=5, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf_words)

print_top_words(nmf,words,4)

[['camera', 'also', 'wesome', 'one'],
 ['quality', 'poor', 'worst', 'working'],
 ['xcellent', 'smartphone', 'cell', 'choic'],
 ['flipkart', 'awesome', 'backup', 'normal'],
 ['loving', 'money', 'awsm', 'worth']]

### NMF - BOW

In [ ]:
bow=CountVectorizer(ngram_range=(1,1))
bow_review=bow.fit_transform(data["review"])
df_bow=pd.DataFrame(bow_review.toarray(),columns=bow.get_feature_names())
bow_words=bow.get_feature_names()
nmf = NMF(n_components=4, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(bow_review)

print_top_words(nmf,bow_words,4)

[['phone', 'battery', 'best', 'camera'],
 ['good', 'camera', 'samsung', 'price'],
 ['product', 'samsung', 'wesome', 'buy'],
 ['nice', 'mobile', 'phone', 'worst']]

In [ ]:
topics=print_top_words(nmf,bow_words,5)
topic=["topic_"+str(i) for i  in range(0,nmf.n_components)]
df_doc_topic=pd.DataFrame(np.round(nmf.transform(df_bow),2),columns=topic)
dominant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic["dominant_topic"] = dominant_topic
df_doc_topic["review"]=dataoriginal["review"]
df_doc_topic["dominant_topic_keywords"]=[topics[i]  for i in dominant_topic]
df_doc_topic

,topic_0,topic_1,topic_2,topic_3,dominant_topic,review,dominant_topic_keywords
0,0.92,0.07,0.00,0.00,0,Bokeh mode is very excellent than any other ha...,"[phone, battery, best, camera, display]"
1,0.28,0.07,0.87,0.00,2,Just awsome product the picture quality is goo...,"[product, samsung, wesome, buy, flipkart]"
2,0.71,0.40,0.00,0.00,0,Samsung galaxy m30s is most useful.But Samsung...,"[phone, battery, best, camera, display]"
3,0.00,0.00,2.05,0.01,2,Not worth for me. i buy for my samsung tv this...,"[product, samsung, wesome, buy, flipkart]"
4,0.88,0.00,0.00,0.00,0,"BSOLUTELY BEST PHONE, GO FOR THIS PHONELET ME ...","[phone, battery, best, camera, display]"
...,...,...,...,...,...,...,...
293,0.04,0.00,0.00,0.00,0,okay,"[phone, battery, best, camera, display]"
294,0.04,0.07,0.06,0.26,3,Super style and good look nice product,"[nice, mobile, phone, worst, delivery]"
295,0.00,0.07,0.00,0.00,1,good,"[good, camera, samsung, price, performance]"
296,0.00,0.07,0.00,0.00,1,good,"[good, camera, samsung, price, performance]"


In [ ]:
np.round(nmf.transform(bow_review),2).shape

(298, 10)

In [ ]:
bow_review.shape

(298, 234)

In [ ]:
nmf.components_.shape

(10, 234)

In [ ]:
topics=print_top_words(nmf,bow_words,)
topic=["topic_"+str(i) for i  in range(0,nmf.n_components)]
df_doc_topic=pd.DataFrame(np.round(nmf.transform(df_bow),2),columns=topic)
dominant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic["dominant_topic"] = dominant_topic
df_doc_topic["review"]=dataoriginal["review"]
df_doc_topic["dominant_topic_keywords"]=[topics[i]  for i in dominant_topic]

#### NMF on topics obtained ------------------------------------------

In [ ]:
### extrating topics
arr=np.array(print_top_words(nmf,bow.get_feature_names(),5)).reshape(1,-1)[0]
bow=CountVectorizer(ngram_range=(1,1))
bow_review=bow.fit_transform(arr)
df_bow=pd.DataFrame(bow_review.toarray(),columns=bow.get_feature_names())
nmf = NMF(n_components=3, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(bow_review)
bow_words=bow.get_feature_names()
print_top_words(nmf,bow_words,4)

[['battery', 'flipkart', 'also', 'power'],
 ['camera', '12', '2020', 'quality'],
 ['buy', '3rd', 'better', 'competition']]

In [ ]:
topics=print_top_words(nmf,bow_words,4)
topic=["topic_"+str(i) for i  in range(0,nmf.n_components)]
df_doc_topic=pd.DataFrame(np.round(nmf.transform(df_bow),2),columns=topic)
dominant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic["dominant_topic"] = dominant_topic
df_doc_topic["review"]=dataoriginal["review"]
df_doc_topic["dominant_topic_keywords"]=[topics[i]  for i in dominant_topic]
df_doc_topic#[["review","dominant_topic_keywords","dominant_topic"]]

,topic_0,topic_1,topic_2,dominant_topic,review,dominant_topic_keywords
0,0.00,0.28,0.00,1,Bokeh mode is very excellent than any other ha...,"[xcellent, value, quality, performance]"
1,0.00,0.00,0.25,2,Just awsome product the picture quality is goo...,"[experience, backup, use, budget]"
2,0.00,0.00,0.25,2,Samsung galaxy m30s is most useful.But Samsung...,"[experience, backup, use, budget]"
3,0.28,0.00,0.00,0,Not worth for me. i buy for my samsung tv this...,"[love, hand, battery, superb]"
4,0.28,0.00,0.00,0,"BSOLUTELY BEST PHONE, GO FOR THIS PHONELET ME ...","[love, hand, battery, superb]"
5,0.00,0.00,0.25,2,I have realized that this phone is full packag...,"[experience, backup, use, budget]"
6,0.28,0.00,0.00,0,"one of the good phone in this price range , th...","[love, hand, battery, superb]"
7,0.00,0.00,0.25,2,Nice mobile and it comes good battery back up....,"[experience, backup, use, budget]"
8,0.00,0.00,0.25,2,Been using this phone for more than 4 months n...,"[experience, backup, use, budget]"
9,0.00,0.28,0.00,1,This phone is works excellent so far after (6 ...,"[xcellent, value, quality, performance]"
